<a href="https://colab.research.google.com/github/victory06/TFG/blob/master/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.io import arff
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Perceptron, Lasso, LogisticRegression, LinearRegression, SGDRegressor, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


In [55]:
"""
Lee las bases de datos y elimina los datos perdidos:
    - path: indica el path que se leerá siempre
    - three_paths: indica si se quieren leer las tres bases de datos o solo una
    - strategy_SI: la estrategia a seguir para quitar los datos perdidos
"""
def readData(path, path2, path3, three_paths=False, strategy_SI = "most_frequent"):
    if(three_paths==False):
      data = arff.loadarff(path)
      dff = pd.DataFrame(data[0])
      dff = dff.apply(LabelEncoder().fit_transform)
      df = dff.to_numpy()[:,:].astype('U13')
      df = df.astype(float)
      imp = SimpleImputer(strategy=strategy_SI)
      df = imp.fit_transform(df)
    else:
      print("Aqui")
      data1 = arff.loadarff(path)
      dff1 = pd.DataFrame(data1[0])
      dff1 = dff1.apply(LabelEncoder().fit_transform)
      df1 = dff1.to_numpy()[:,:].astype('U13')
      df1 = df1.astype(float)

      data2 = arff.loadarff(path)
      dff2 = pd.DataFrame(data2[0])
      dff2 = dff2.apply(LabelEncoder().fit_transform)
      df2 = dff2.to_numpy()[:,:].astype('U13')
      df2 = df2.astype(float)

      data3 = arff.loadarff(path)
      dff3 = pd.DataFrame(data3[0])
      dff3 = dff3.apply(LabelEncoder().fit_transform)
      df3 = dff3.to_numpy()[:,:].astype('U13')
      df3 = df3.astype(float)

      df = np.concatenate((df1,df2,df3),axis=0)
      np.random.shuffle(df)
      imp = SimpleImputer(strategy=strategy_SI)
      df = imp.fit_transform(df)

    return df[:, :-1], df[:, -1], df

    

In [56]:
x, y, df = readData('/content/drive/MyDrive/TFG/ChildASD/Autism-Child-Data.arff','/content/drive/MyDrive/TFG/AdolescentASD/Autism-Adolescent-Data.arff','/content/drive/MyDrive/TFG/AdultASD/Autism-Adult-Data.arff',three_paths=True)
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.30, random_state=1)
df

Aqui


array([[1., 0., 1., ..., 0., 2., 1.],
       [1., 1., 1., ..., 0., 2., 1.],
       [1., 1., 1., ..., 0., 2., 1.],
       ...,
       [1., 1., 1., ..., 0., 2., 1.],
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 1., 1., ..., 0., 2., 1.]])

In [58]:
preproc=[("var", VarianceThreshold(0.01)),
        ("standardize", StandardScaler()),
        ("poly", PolynomialFeatures(2))]

pipe=Pipeline(preproc + [('model', SGDRegressor())])

#pipe=Pipeline([('lr', LogisticRegression())])

In [59]:

params_grid=[
        {"model":[SGDRegressor(max_iter=500)],
               "model__loss":['huber', 'squared_loss', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
               "model__penalty":['l1','l2'],
               "model__alpha":np.logspace(-5,5,5),
               "poly__degree":[1,2]},
        {"model":[LinearRegression()],
               "poly__degree":[1,2]},
        {"model":[Ridge()],
               "poly__degree":[1,2],
               "model__alpha":np.logspace(-5,5,5)},
        {"model":[Lasso()],
               "poly__degree":[1,2],
               "model__alpha":np.logspace(-5,5,5)}]
"""
params_grid=[
        {"lr":[LogisticRegression(penalty='l1',max_iter=500)],
                "lr__C":np.logspace(-2,2,5),
                "lr__solver":['lbfgs']},
       {"lr": [RandomForestClassifier(random_state = 1,
                                       n_jobs = -1, criterion = 'entropy')],
         "lr__n_estimators": [100, 200],
         "lr__max_depth": [6, 8]},
        {"lr": [SVC(kernel='rbf', gamma='scale', max_iter=1000, degree=2)],
               "lr__C":np.logspace(-2,2,5)},
        {"lr": [MLPClassifier(random_state=1, max_iter=1000, hidden_layer_sizes=60)],
           "lr__solver":['sgd'],
           "lr__activation":['relu','logistic']}
            
]
"""

'\nparams_grid=[\n        {"lr":[LogisticRegression(penalty=\'l1\',max_iter=500)],\n                "lr__C":np.logspace(-2,2,5),\n                "lr__solver":[\'lbfgs\']},\n       {"lr": [RandomForestClassifier(random_state = 1,\n                                       n_jobs = -1, criterion = \'entropy\')],\n         "lr__n_estimators": [100, 200],\n         "lr__max_depth": [6, 8]},\n        {"lr": [SVC(kernel=\'rbf\', gamma=\'scale\', max_iter=1000, degree=2)],\n               "lr__C":np.logspace(-2,2,5)},\n        {"lr": [MLPClassifier(random_state=1, max_iter=1000, hidden_layer_sizes=60)],\n           "lr__solver":[\'sgd\'],\n           "lr__activation":[\'relu\',\'logistic\']}\n            \n]\n'

In [60]:
best_model=GridSearchCV(pipe,params_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)
best_model.fit(x_train,y_train)
print("Parámetros del mejor clasificador:\n{}".format(best_model.best_params_))
print("Error en CV: {:0.3f}%".format(100.0 * best_model.best_score_))
print("Error en training: {:0.3f}%".format(
        100.0 * best_model.score(x_train, y_train)))
print("Error en test: {:0.3f}%".format(
        100.0 * best_model.score(x_test, y_test)))


Fitting 5 folds for each of 102 candidates, totalling 510 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Parámetros del mejor clasificador:
{'model': Lasso(alpha=0.0031622776601683794, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False), 'model__alpha': 0.0031622776601683794, 'poly__degree': 2}
Error en CV: -5.189%
Error en training: -2.899%
Error en test: -3.890%


[Parallel(n_jobs=-1)]: Done 510 out of 510 | elapsed:    4.0s finished
